## Import libraries

In [1]:
import pandas as pd
import numpy as np

## Import KRIs data from Excel into dataframes

In [2]:
#KRI Values
df_KRI_values = pd.read_excel('Risk Data Set for Modeling - Updated 11.5.2020(1227).xlsx', sheet_name = 'KRI Values')

#KRI Type Rating
df_KRI_type_rating = pd.read_excel('Risk Data Set for Modeling - Updated 11.5.2020(1227).xlsx', sheet_name = 'KRI Type and Rating')

#Find common KRIs
KRI_val = list(df_KRI_values['KRI Code'].unique())
KRI_rating = list(df_KRI_type_rating['KRI Code'].unique())

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

KRI_com = intersection(KRI_val, KRI_rating)
len(KRI_com)

#Merge two dataframes
df_KRI_values_1 = df_KRI_values.loc[df_KRI_values['KRI Code'].isin(KRI_com)]
df_KRI_type_rating_1 = df_KRI_type_rating.loc[df_KRI_type_rating['KRI Code'].isin(KRI_com)]
df_KRI_type_rating_1 = df_KRI_type_rating_1.drop_duplicates(['KRI Code', 'Risk Type', 'Severity Rating'])

df_KRIs = pd.merge(df_KRI_values_1, df_KRI_type_rating_1, on = 'KRI Code', how='inner')

### Add Data for Nov 2020 for Home Loans (same as Nov 2018) but with all KRIs having Value Score = Green and Severity Rating a mix of Negligible and Moderate

This was done to add some data where Area average is less than 10th percentile, for demo purposes.

In [3]:
df_nov18 = df_KRIs.loc[df_KRIs['Submission Month'] == 'Nov-2018']
df_nov18 = df_nov18.loc[df_nov18['Area Name'] == 'Home Loans']
df_nov18['Submission Month'] = 'Nov-2020'
df_nov18.loc[df_nov18['Value Score'] == 'Amber', 'Severity Rating'] = 'Moderate'
df_nov18.loc[df_nov18['Value Score'] == 'Red', 'Severity Rating'] = 'Moderate'
df_nov18.loc[df_nov18['Value Score'] == 'Green', 'Severity Rating'] = 'Negligible'
df_nov18['Value Score'] = 'Green'
df_nov18

,Submission Month,KRI Code,Value Score,Unit,Frequency,Area Name,Risk Type,Severity Rating,Unnamed: 3
1285,Nov-2020,CI28659,Green,Number,Monthly,Home Loans,Operational,Negligible,NaN
1309,Nov-2020,CI28660,Green,Percentage,Monthly,Home Loans,Reputational,Negligible,NaN
1334,Nov-2020,CI28661,Green,Number,Monthly,Home Loans,Operational,Negligible,NaN
1358,Nov-2020,CI28662,Green,Percentage,Monthly,Home Loans,Operational,Moderate,NaN
1382,Nov-2020,CI28663,Green,Number,Monthly,Home Loans,Operational,Negligible,NaN
1406,Nov-2020,CI28665,Green,Percentage,Monthly,Home Loans,Operational,Moderate,NaN
1430,Nov-2020,CI28666,Green,Percentage,Monthly,Home Loans,Compliance,Negligible,NaN
1454,Nov-2020,CI28667,Green,Percentage,Monthly,Home Loans,Compliance,Negligible,NaN
1478,Nov-2020,CI28669,Green,Number,Monthly,Home Loans,Operational,Moderate,NaN
1502,Nov-2020,CI28670,Green,Number,Monthly,Home Loans,Compliance,Negligible,NaN


In [4]:
df_KRIs = pd.concat([df_KRIs, df_nov18])
df_KRIs.head()

,Submission Month,KRI Code,Value Score,Unit,Frequency,Area Name,Risk Type,Severity Rating,Unnamed: 3
0,Jun-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN
1,Jul-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN
2,Aug-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN
3,Sep-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN
4,Oct-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN


In [5]:
df_KRIs['Area Name'].nunique()

13

In [6]:
df_KRIs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4004 entries, 0 to 1830
Data columns (total 9 columns):
Submission Month    4004 non-null object
KRI Code            4004 non-null object
Value Score         4004 non-null object
Unit                4004 non-null object
Frequency           4004 non-null object
Area Name           4004 non-null object
Risk Type           4004 non-null object
Severity Rating     4004 non-null object
Unnamed: 3          68 non-null float64
dtypes: float64(1), object(8)
memory usage: 312.8+ KB


## Create IDs for Year, Month, Quarter

Data has Submission month which is an object field. Creating the columns for Year_ID, Month_ID, Quarter_ID that would aid the model and analysis.

In [7]:
#Month IDs, Year IDs, Quarter IDs
month = {'Jun-2020': 202006, 'Jul-2020': 202007, 'Aug-2020':202008, 
         'Sep-2020':202009, 'Oct-2020':202010, 'Nov-2020':202011,
       'Jul-2019':201907, 'Jun-2019':201906, 'May-2019':201905, 'Apr-2019':201904, 'Mar-2019':201903,
       'Feb-2019':201902, 'Jan-2019':201901, 'Aug-2019':201908, 'Sep-2019':201909, 'Oct-2019':201910,
       'Nov-2019':201911, 'Dec-2019':201912, 'Jan-2020':202001, 'Feb-2020':202002, 'Mar-2020':202003,
       'Apr-2020':202004, 'May-2020':202005, 'Jan-2018':201801, 'Feb-2018':201802, 'Mar-2018':201803,
       'Apr-2018':201804, 'May-2018':201805, 'Jun-2018':201806, 'Jul-2018':201807, 'Aug-2018':201808,
       'Sep-2018':201809, 'Oct-2018':201810, 'Nov-2018':201811, 'Dec-2018':201812}

quarter = {'Jun-2020': '2020-Q2', 'Jul-2020': '2020-Q3', 'Aug-2020':'2020-Q3', 
         'Sep-2020':'2020-Q3', 'Oct-2020':'2020-Q4', 'Nov-2020':'2020-Q4',
       'Jul-2019':'2019-Q3', 'Jun-2019':'2019-Q2', 'May-2019':'2019-Q2', 'Apr-2019':'2019-Q2', 'Mar-2019':'2019-Q1',
       'Feb-2019':'2019-Q1', 'Jan-2019':'2019-Q1', 'Aug-2019':'2019-Q3', 'Sep-2019':'2019-Q3', 'Oct-2019':'2019-Q4',
       'Nov-2019':'2019-Q4', 'Dec-2019':'2019-Q4', 'Jan-2020':'2020-Q1', 'Feb-2020':'2020-Q1', 'Mar-2020':'2020-Q1',
       'Apr-2020':'2020-Q2', 'May-2020':'2020-Q2', 'Jan-2018':'2018-Q1', 'Feb-2018':'2018-Q1', 'Mar-2018':'2018-Q1',
       'Apr-2018':'2018-Q2', 'May-2018':'2018-Q2', 'Jun-2018':'2018-Q2', 'Jul-2018':'2018-Q3', 'Aug-2018':'2018-Q3',
       'Sep-2018':'2018-Q3', 'Oct-2018':'2018-Q4', 'Nov-2018':'2018-Q4', 'Dec-2018':'2018-Q4'}

year = {'Jun-2020': 2020, 'Jul-2020': 2020, 'Aug-2020':2020, 
         'Sep-2020':2020, 'Oct-2020':2020, 'Nov-2020':2020,
       'Jul-2019':2019, 'Jun-2019':2019, 'May-2019':2019, 'Apr-2019':2019, 'Mar-2019':2019,
       'Feb-2019':2019, 'Jan-2019':2019, 'Aug-2019':2019, 'Sep-2019':2019, 'Oct-2019':2019,
       'Nov-2019':2019, 'Dec-2019':2019, 'Jan-2020':2020, 'Feb-2020':2020, 'Mar-2020':2020,
       'Apr-2020':2020, 'May-2020':2020, 'Jan-2018':2018, 'Feb-2018':2018, 'Mar-2018':2018,
       'Apr-2018':2018, 'May-2018':2018, 'Jun-2018':2018, 'Jul-2018':2018, 'Aug-2018':2018,
       'Sep-2018':2018, 'Oct-2018':2018, 'Nov-2018':2018, 'Dec-2018':2018}

#Add Month, Quarter and Year IDs to the dataframe
df_KRIs['Month_ID'] = df_KRIs['Submission Month'].replace(month)
df_KRIs['Quarter_ID'] = df_KRIs['Submission Month'].replace(quarter)
df_KRIs['Year_ID'] = df_KRIs['Submission Month'].replace(year)

In [8]:
df_KRIs.head()

,Submission Month,KRI Code,Value Score,Unit,Frequency,Area Name,Risk Type,Severity Rating,Unnamed: 3,Month_ID,Quarter_ID,Year_ID
0,Jun-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202006,2020-Q2,2020
1,Jul-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202007,2020-Q3,2020
2,Aug-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202008,2020-Q3,2020
3,Sep-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202009,2020-Q3,2020
4,Oct-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202010,2020-Q4,2020


## Assign the weights computed using AHP to Value score and severity rating

In [9]:
#Weights using AHP
#Green = 0.16, Amber = 0.32, Red = 0.52, 
#Critical = 0.43, High = 0.32, Moderate = 0.18, Negligible = 0.08
AHP_rating_wt = {'Critical': 0.43, 'High': 0.32, 'Moderate': 0.18, 'Negligible': 0.08}
AHP_score_wt = {'Green': 0.16, 'Amber': 0.32, 'Red': 0.52}

#AHP Dataframe
df_KRIs_wt_AHP = df_KRIs.copy(deep=True)
df_KRIs_wt_AHP['AHP_Rating_wt'] = df_KRIs_wt_AHP['Severity Rating'].replace(AHP_rating_wt)
df_KRIs_wt_AHP['AHP_Score_wt'] = df_KRIs_wt_AHP['Value Score'].replace(AHP_score_wt)
df_KRIs_wt_AHP[['AHP_Rating_wt', 'AHP_Score_wt']] = df_KRIs_wt_AHP[['AHP_Rating_wt', 'AHP_Score_wt']].astype(float)

#Function for Rank
def rank(row):
    if row['Value Score'] == "Green":
        val = abs(row['AHP_Score_wt'] - row['AHP_Rating_wt'])
    else:
        val = row['AHP_Score_wt'] + row['AHP_Rating_wt']
    return val

#Calculate Rank
df_KRIs_wt_AHP['AHP_Rank'] = 1
df_KRIs_wt_AHP['AHP_Rank'] = df_KRIs_wt_AHP.apply(rank, axis=1)
df_KRIs_wt_AHP.head()

,Submission Month,KRI Code,Value Score,Unit,Frequency,Area Name,Risk Type,Severity Rating,Unnamed: 3,Month_ID,Quarter_ID,Year_ID,AHP_Rating_wt,AHP_Score_wt,AHP_Rank
0,Jun-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202006,2020-Q2,2020,0.32,0.16,0.16
1,Jul-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202007,2020-Q3,2020,0.32,0.16,0.16
2,Aug-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202008,2020-Q3,2020,0.32,0.16,0.16
3,Sep-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202009,2020-Q3,2020,0.32,0.16,0.16
4,Oct-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202010,2020-Q4,2020,0.32,0.16,0.16


## Combinations for various parameters
#### Value Score

In [10]:
r_head = ['Critical', 'High', 'Moderate', 'Negligible']
r_val = [0.43, 0.32, 0.18, 0.08]
d_r = {'header_r':r_head,'values_r':r_val}
r_df = pd.DataFrame(d_r)
r_df

,header_r,values_r
0,Critical,0.43
1,High,0.32
2,Moderate,0.18
3,Negligible,0.08


 #### Severity Rating

In [11]:
s_head = ['Green', 'Amber', 'Red']
s_val = [0.16, 0.32, 0.52]
d_s = {'header_s':s_head,'values_s':s_val}
s_df = pd.DataFrame(d_s)
s_df

,header_s,values_s
0,Green,0.16
1,Amber,0.32
2,Red,0.52


#### Combinations

In [13]:
s_df['key'] = 1
r_df['key'] = 1
combo = pd.merge(r_df, s_df, on ='key').drop("key", 1) 
combo['label'] = combo['header_s'] + '_' + combo['header_r']

# Calculate Total
for i, r in combo.iterrows():
  if combo.at[i, "header_s"] == 'Green':
    combo.at[i,"total"] = abs(combo.at[i,"values_s"] - combo.at[i,"values_r"]) #For Green, Total = value score - severity rating
  else:
    combo.at[i,"total"] = combo.at[i,'values_s'] + combo.at[i,'values_r']

# Sort the values
combo = combo.sort_values(by = ['total'])
combo['grand_total'] = combo['total'].sum()
combo['norm'] = combo['total']/combo['grand_total']
combo = combo[['label', 'total', 'norm']]
combo = combo.rename(columns={'total' : 'AHP_Rank'})


combo

,label,AHP_Rank,norm
6,Green_Moderate,0.02,0.003384
9,Green_Negligible,0.08,0.013536
3,Green_High,0.16,0.027073
0,Green_Critical,0.27,0.045685
10,Amber_Negligible,0.40,0.067682
7,Amber_Moderate,0.50,0.084602
11,Red_Negligible,0.60,0.101523
4,Amber_High,0.64,0.108291
8,Red_Moderate,0.70,0.118443
1,Amber_Critical,0.75,0.126904


In [14]:
# Export Combo to Excel

#combo.to_excel(r'/content/drive/MyDrive/Santander/Data_Rank_Avg/combo.xlsx', index_label="index")

## Assessment benchmarks

#### 10th and 20th Percentile Values

In [15]:
print('Threshold for 10%:', combo.norm.quantile(0.10))
print('Threshold for 20%:', combo.norm.quantile(0.20))

Threshold for 10%: 0.014890016920473773
Threshold for 20%: 0.030795262267343488


##### Adding the 10th and 20th Percentile values as columns to the DataFrame and join with Combo dataframe

In [16]:
df_KRIs_wt_AHP = pd.merge(df_KRIs_wt_AHP, combo, how = 'left', on = ['AHP_Rank'])
df_KRIs_wt_AHP['10th_percentile'] = 0.01489
df_KRIs_wt_AHP['20th_percentile'] = 0.03079
df_KRIs_wt_AHP.head()

,Submission Month,KRI Code,Value Score,Unit,Frequency,Area Name,Risk Type,Severity Rating,Unnamed: 3,Month_ID,Quarter_ID,Year_ID,AHP_Rating_wt,AHP_Score_wt,AHP_Rank,label,norm,10th_percentile,20th_percentile
0,Jun-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202006,2020-Q2,2020,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079
1,Jul-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202007,2020-Q3,2020,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079
2,Aug-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202008,2020-Q3,2020,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079
3,Sep-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202009,2020-Q3,2020,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079
4,Oct-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202010,2020-Q4,2020,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079


## Averages for KRIs:

For a Submission Month, for an Area:
Area Average = Total of Avg. for all KRIs / count of KRIs

In [17]:
df_KRIs_wt_AHP['month_total'] = df_KRIs_wt_AHP.groupby(['Area Name','Month_ID'])['norm'].transform('sum')
df_KRIs_wt_AHP['mo_KRI_count'] = df_KRIs_wt_AHP.groupby(['Area Name','Month_ID'])['KRI Code'].transform('nunique')
df_KRIs_wt_AHP['mo_avg'] = df_KRIs_wt_AHP['month_total']/df_KRIs_wt_AHP['mo_KRI_count']
df_KRIs_wt_AHP.head()

,Submission Month,KRI Code,Value Score,Unit,Frequency,Area Name,Risk Type,Severity Rating,Unnamed: 3,Month_ID,...,AHP_Rating_wt,AHP_Score_wt,AHP_Rank,label,norm,10th_percentile,20th_percentile,month_total,mo_KRI_count,mo_avg
0,Jun-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202006,...,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079,0.027073,1,0.027073
1,Jul-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202007,...,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079,0.027073,1,0.027073
2,Aug-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202008,...,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079,0.027073,1,0.027073
3,Sep-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202009,...,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079,0.027073,1,0.027073
4,Oct-2020,CI28824,Green,Percentage,Monthly,Analytics & Decision Science,Compliance,High,NaN,202010,...,0.32,0.16,0.16,Green_High,0.027073,0.01489,0.03079,0.027073,1,0.027073


### Export the KRIs data in an excel file:

In [19]:
#df_KRIs_wt_AHP.to_excel('KRIs_AHP_Rank_Avg.xlsx', index_label="index")

## Import Issue Management Data:

In [34]:
#Issue Management
df_IM = pd.read_excel('IM_Data_1.22.21.xlsx')
df_IM.head()

,AD Due Date by Month,Area,Department,AD ID,AD Target Due Date,AD Status,Priority Rating,Resolution Status,Current Target Date,Number of MA,Date of Creation,Products,Regulatory Hashtags,Vendor,Application,Issues opened by Month,Issues Closed/Abandoned by Month
0,Apr-21,Business Banking,NaN,005056A0614D1EDB85F8030A82F98364,2021-04-01,On Track,Moderate,Remediate,2021-04-01,1.0,2020-10-26,No,Yes,No,No,Oct-20,NaN
1,Complete,Consumer Lending,Home Loans,005056A0614D1ED8BCBBD2D9FAE5230B,2019-08-30,Complete,Moderate,Remediate,2019-08-30,4.0,2018-11-27,Yes,Yes,No,No,Created 13 Months Ago,Closed 13 Months Ago
2,Complete,Consumer Lending,Home Loans,005056A0614D1EDA96B3BDE076BEE563,2020-04-30,Complete,Moderate,Remediate,2020-04-30,1.0,2020-02-27,Yes,Yes,Yes,Yes,Feb-20,Apr-20
3,Complete,Operations,Payments,005056A0614D1EE8AEBF72AD6B4A0EED,2020-12-31,Complete,Moderate,Remediate,2020-12-31,1.0,2018-09-16,Yes,No,No,Yes,Created 13 Months Ago,Dec-20
4,Complete,FLOD & Bus. Controls,NaN,005056A008D11EDAA3DA9DCAF13A707E,2020-11-30,Complete,Low,Remediate,2020-11-30,3.0,2020-05-05,Yes,No,No,No,May-20,Dec-20


In [35]:
df_IM['Area'].nunique()

12

In [36]:
df_IM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3180 entries, 0 to 3179
Data columns (total 17 columns):
AD Due Date by Month                3180 non-null object
Area                                3180 non-null object
Department                          2341 non-null object
AD ID                               3180 non-null object
AD Target Due Date                  3180 non-null datetime64[ns]
AD Status                           3180 non-null object
Priority Rating                     3180 non-null object
Resolution Status                   3180 non-null object
Current Target Date                 3113 non-null datetime64[ns]
Number of MA                        3114 non-null float64
Date of Creation                    3180 non-null datetime64[ns]
Products                            3180 non-null object
Regulatory Hashtags                 3180 non-null object
Vendor                              3180 non-null object
Application                         3180 non-null object
Issues opened

Modeling Dataframe columns:

In [37]:
df_IM['year'] = pd.DatetimeIndex(df_IM['Date of Creation']).year
df_IM['month'] = pd.DatetimeIndex(df_IM['Date of Creation']).month

df_IM[['year', 'month']] = df_IM[['year', 'month']].astype(str)

# Append '0' before the month ID for 1-9 to make month uniform with 2 characters
month_dict = {'1': '01','2': '02', '3': '03', '4': '04', '5': '05', '6': '06','7': '07','8': '08','9': '09'} 
df_IM['month'] = df_IM['month'].replace(month_dict)

# Concatenate year-month
df_IM['sub_month'] = df_IM.year + df_IM.month

# Convert sub_month into integer
df_IM['sub_month'] = df_IM['sub_month'].astype(int)

# Replace blanks with string N/A in department column
df_IM['Department'] = df_IM['Department'].fillna('N/A')

df_IM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3180 entries, 0 to 3179
Data columns (total 20 columns):
AD Due Date by Month                3180 non-null object
Area                                3180 non-null object
Department                          3180 non-null object
AD ID                               3180 non-null object
AD Target Due Date                  3180 non-null datetime64[ns]
AD Status                           3180 non-null object
Priority Rating                     3180 non-null object
Resolution Status                   3180 non-null object
Current Target Date                 3113 non-null datetime64[ns]
Number of MA                        3114 non-null float64
Date of Creation                    3180 non-null datetime64[ns]
Products                            3180 non-null object
Regulatory Hashtags                 3180 non-null object
Vendor                              3180 non-null object
Application                         3180 non-null object
Issues opened

In [38]:
df_IM.head()

,AD Due Date by Month,Area,Department,AD ID,AD Target Due Date,AD Status,Priority Rating,Resolution Status,Current Target Date,Number of MA,Date of Creation,Products,Regulatory Hashtags,Vendor,Application,Issues opened by Month,Issues Closed/Abandoned by Month,year,month,sub_month
0,Apr-21,Business Banking,N/A,005056A0614D1EDB85F8030A82F98364,2021-04-01,On Track,Moderate,Remediate,2021-04-01,1.0,2020-10-26,No,Yes,No,No,Oct-20,NaN,2020,10,202010
1,Complete,Consumer Lending,Home Loans,005056A0614D1ED8BCBBD2D9FAE5230B,2019-08-30,Complete,Moderate,Remediate,2019-08-30,4.0,2018-11-27,Yes,Yes,No,No,Created 13 Months Ago,Closed 13 Months Ago,2018,11,201811
2,Complete,Consumer Lending,Home Loans,005056A0614D1EDA96B3BDE076BEE563,2020-04-30,Complete,Moderate,Remediate,2020-04-30,1.0,2020-02-27,Yes,Yes,Yes,Yes,Feb-20,Apr-20,2020,02,202002
3,Complete,Operations,Payments,005056A0614D1EE8AEBF72AD6B4A0EED,2020-12-31,Complete,Moderate,Remediate,2020-12-31,1.0,2018-09-16,Yes,No,No,Yes,Created 13 Months Ago,Dec-20,2018,09,201809
4,Complete,FLOD & Bus. Controls,N/A,005056A008D11EDAA3DA9DCAF13A707E,2020-11-30,Complete,Low,Remediate,2020-11-30,3.0,2020-05-05,Yes,No,No,No,May-20,Dec-20,2020,05,202005


### AHP Weights for IM:

In [39]:
#Weights using AHP

# For AD Status:
#No Mitigating Actions = 0.20, Complete = 0.11, On Track = 0.14, At Risk = 0.23, Past Due = 0.32

# For Priority Rating:
#Critical = 0.43, High = 0.32, Moderate = 0.18, Negligible = 0.08

# Dictionary for AD Status:
AHP_AD_dict = {'No Mitigating Actions': 0.20, 'Complete': 0.11, 'On Track': 0.14, 'At Risk': 0.23, 'Past Due': 0.32}

#Dictionary for PR:
AHP_PR_dict = {'Critical': 0.43, 'High': 0.32, 'Moderate': 0.18, 'Low': 0.08}

#AHP Dataframe
df_IM_wt_AHP = df_IM.copy(deep=True)
df_IM_wt_AHP['AHP_AD_wt'] = df_IM_wt_AHP['AD Status'].replace(AHP_AD_dict)
df_IM_wt_AHP['AHP_PR_wt'] = df_IM_wt_AHP['Priority Rating'].replace(AHP_PR_dict)
df_IM_wt_AHP[['AHP_AD_wt', 'AHP_PR_wt']] = df_IM_wt_AHP[['AHP_AD_wt', 'AHP_PR_wt']].astype(float)

#AHP Rank based on condition for Value score
for i, r in df_IM_wt_AHP.iterrows():
  if ((df_IM_wt_AHP.at[i, "AD Status"] == 'Complete') | (df_IM_wt_AHP.at[i, "AD Status"] == 'On Track')):
    df_IM_wt_AHP.at[i,"AHP_IM_Rank"] = abs(df_IM_wt_AHP.at[i,"AHP_AD_wt"] - df_IM_wt_AHP.at[i,"AHP_PR_wt"])
  else:
    df_IM_wt_AHP.at[i,"AHP_IM_Rank"] = df_IM_wt_AHP.at[i,"AHP_AD_wt"] + df_IM_wt_AHP.at[i,"AHP_PR_wt"]
df_IM_wt_AHP.head()

,AD Due Date by Month,Area,Department,AD ID,AD Target Due Date,AD Status,Priority Rating,Resolution Status,Current Target Date,Number of MA,...,Vendor,Application,Issues opened by Month,Issues Closed/Abandoned by Month,year,month,sub_month,AHP_AD_wt,AHP_PR_wt,AHP_IM_Rank
0,Apr-21,Business Banking,N/A,005056A0614D1EDB85F8030A82F98364,2021-04-01,On Track,Moderate,Remediate,2021-04-01,1.0,...,No,No,Oct-20,NaN,2020,10,202010,0.14,0.18,0.04
1,Complete,Consumer Lending,Home Loans,005056A0614D1ED8BCBBD2D9FAE5230B,2019-08-30,Complete,Moderate,Remediate,2019-08-30,4.0,...,No,No,Created 13 Months Ago,Closed 13 Months Ago,2018,11,201811,0.11,0.18,0.07
2,Complete,Consumer Lending,Home Loans,005056A0614D1EDA96B3BDE076BEE563,2020-04-30,Complete,Moderate,Remediate,2020-04-30,1.0,...,Yes,Yes,Feb-20,Apr-20,2020,02,202002,0.11,0.18,0.07
3,Complete,Operations,Payments,005056A0614D1EE8AEBF72AD6B4A0EED,2020-12-31,Complete,Moderate,Remediate,2020-12-31,1.0,...,No,Yes,Created 13 Months Ago,Dec-20,2018,09,201809,0.11,0.18,0.07
4,Complete,FLOD & Bus. Controls,N/A,005056A008D11EDAA3DA9DCAF13A707E,2020-11-30,Complete,Low,Remediate,2020-11-30,3.0,...,No,No,May-20,Dec-20,2020,05,202005,0.11,0.08,0.03


### Issue management combos:

##### AD Status

In [40]:
ad_head = ['No Mitigating Actions', 'Complete', 'On Track', 'At Risk', 'Past Due']
ad_val = [0.20, 0.11, 0.14, 0.23, 0.32]
d_ad = {'header_ad':ad_head,'values_ad':ad_val}
ad_df = pd.DataFrame(d_ad)
ad_df

,header_ad,values_ad
0,No Mitigating Actions,0.20
1,Complete,0.11
2,On Track,0.14
3,At Risk,0.23
4,Past Due,0.32


##### Priority Rating

In [41]:
pr_head = ['Critical', 'High', 'Moderate', 'Low']
pr_val = [0.43, 0.32, 0.18, 0.08]
d_pr = {'header_pr':pr_head,'values_pr':pr_val}
pr_df = pd.DataFrame(d_pr)
pr_df

,header_pr,values_pr
0,Critical,0.43
1,High,0.32
2,Moderate,0.18
3,Low,0.08


##### Combinations

In [42]:
ad_df['key'] = 1
pr_df['key'] = 1
combo_im = pd.merge(ad_df, pr_df, on ='key')
combo_im['label'] = combo_im['header_ad'] + '_' + combo_im['header_pr']

# Calculate Total
for i, r in combo_im.iterrows():
  if ((combo_im.at[i, "header_ad"] == 'Complete') | (combo_im.at[i, "header_ad"] == 'On Track')):
    combo_im.at[i,"total"] = abs(combo_im.at[i,"values_ad"] - combo_im.at[i,"values_pr"]) #For Complete & On Track, Total = value score - severity rating
  else:
    combo_im.at[i,'total'] = combo_im.at[i,'values_ad'] + combo_im.at[i,'values_pr']
combo_im = combo_im.sort_values(by = ['total'])

# Get norm values
combo_im['grand_total'] = combo_im['total'].sum()
combo_im['norm'] = combo_im['total']/combo_im['grand_total']

# Rearrange columns
combo_im = combo_im[['label', 'total', 'norm']]

#Rename rank column
combo_im = combo_im.rename(columns={'total' : 'AHP_IM_Rank'})

combo_im

,label,AHP_IM_Rank,norm
7,Complete_Low,0.03,0.004149
10,On Track_Moderate,0.04,0.005533
11,On Track_Low,0.06,0.008299
6,Complete_Moderate,0.07,0.009682
9,On Track_High,0.18,0.024896
5,Complete_High,0.21,0.029046
3,No Mitigating Actions_Low,0.28,0.038728
8,On Track_Critical,0.29,0.040111
15,At Risk_Low,0.31,0.042877
4,Complete_Critical,0.32,0.044260


In [29]:
# Export Combos for IM to Excel
#combo_im.to_excel(r'/content/drive/MyDrive/Santander/Data_Rank_Avg/combo_im.xlsx', index_label="index")

## Assessment benchmarks for IM

#### 10th and 20th Percentile Values

In [43]:
print('Threshold for 10%:', combo_im.norm.quantile(0.1))
print('Threshold for 20%:', combo_im.norm.quantile(0.2))

Threshold for 10%: 0.008022130013831261
Threshold for 20%: 0.021853388658367914


##### Adding the 10th and 20th Percentile values as columns to the IM DataFrame and join with Combo dataframe

In [44]:
df_IM_wt_AHP['10th_percentile'] = 0.00802
df_IM_wt_AHP['20th_percentile'] = 0.02185
df_IM_wt_AHP = pd.merge(df_IM_wt_AHP, combo_im, how = 'left', on = ['AHP_IM_Rank'])
df_IM_wt_AHP.head()

,AD Due Date by Month,Area,Department,AD ID,AD Target Due Date,AD Status,Priority Rating,Resolution Status,Current Target Date,Number of MA,...,year,month,sub_month,AHP_AD_wt,AHP_PR_wt,AHP_IM_Rank,10th_percentile,20th_percentile,label,norm
0,Apr-21,Business Banking,N/A,005056A0614D1EDB85F8030A82F98364,2021-04-01,On Track,Moderate,Remediate,2021-04-01,1.0,...,2020,10,202010,0.14,0.18,0.04,0.00802,0.02185,On Track_Moderate,0.005533
1,Complete,Consumer Lending,Home Loans,005056A0614D1ED8BCBBD2D9FAE5230B,2019-08-30,Complete,Moderate,Remediate,2019-08-30,4.0,...,2018,11,201811,0.11,0.18,0.07,0.00802,0.02185,Complete_Moderate,0.009682
2,Complete,Consumer Lending,Home Loans,005056A0614D1EDA96B3BDE076BEE563,2020-04-30,Complete,Moderate,Remediate,2020-04-30,1.0,...,2020,02,202002,0.11,0.18,0.07,0.00802,0.02185,Complete_Moderate,0.009682
3,Complete,Operations,Payments,005056A0614D1EE8AEBF72AD6B4A0EED,2020-12-31,Complete,Moderate,Remediate,2020-12-31,1.0,...,2018,09,201809,0.11,0.18,0.07,0.00802,0.02185,Complete_Moderate,0.009682
4,Complete,FLOD & Bus. Controls,N/A,005056A008D11EDAA3DA9DCAF13A707E,2020-11-30,Complete,Low,Remediate,2020-11-30,3.0,...,2020,05,202005,0.11,0.08,0.03,0.00802,0.02185,Complete_Low,0.004149


## Averages for IM:

For a Submission Month, for an Area:
Area Average = Total of Avg. for all KRIs / count of KRIs

In [45]:
# For Area
df_IM_wt_AHP['area_month_total'] = df_IM_wt_AHP.groupby(['Area', 'sub_month'])['norm'].transform('sum')
df_IM_wt_AHP['area_mo_issue_count'] = df_IM_wt_AHP.groupby(['Area','sub_month'])['AD ID'].transform('nunique')
df_IM_wt_AHP['area_mo_avg'] = df_IM_wt_AHP['area_month_total']/df_IM_wt_AHP['area_mo_issue_count']

# For Department
df_IM_wt_AHP['dept_month_total'] = df_IM_wt_AHP.groupby(['Area', 'Department','sub_month'])['norm'].transform('sum')
df_IM_wt_AHP['dept_mo_issue_count'] = df_IM_wt_AHP.groupby(['Area', 'Department','sub_month'])['AD ID'].transform('nunique')
df_IM_wt_AHP['dept_mo_avg'] = df_IM_wt_AHP['dept_month_total']/df_IM_wt_AHP['dept_mo_issue_count']

df_IM_wt_AHP.head()

,AD Due Date by Month,Area,Department,AD ID,AD Target Due Date,AD Status,Priority Rating,Resolution Status,Current Target Date,Number of MA,...,10th_percentile,20th_percentile,label,norm,area_month_total,area_mo_issue_count,area_mo_avg,dept_month_total,dept_mo_issue_count,dept_mo_avg
0,Apr-21,Business Banking,N/A,005056A0614D1EDB85F8030A82F98364,2021-04-01,On Track,Moderate,Remediate,2021-04-01,1.0,...,0.00802,0.02185,On Track_Moderate,0.005533,0.071923,4,0.017981,0.005533,1,0.005533
1,Complete,Consumer Lending,Home Loans,005056A0614D1ED8BCBBD2D9FAE5230B,2019-08-30,Complete,Moderate,Remediate,2019-08-30,4.0,...,0.00802,0.02185,Complete_Moderate,0.009682,0.372061,43,0.008653,0.260028,28,0.009287
2,Complete,Consumer Lending,Home Loans,005056A0614D1EDA96B3BDE076BEE563,2020-04-30,Complete,Moderate,Remediate,2020-04-30,1.0,...,0.00802,0.02185,Complete_Moderate,0.009682,0.344398,24,0.014350,0.211618,17,0.012448
3,Complete,Operations,Payments,005056A0614D1EE8AEBF72AD6B4A0EED,2020-12-31,Complete,Moderate,Remediate,2020-12-31,1.0,...,0.00802,0.02185,Complete_Moderate,0.009682,1.387275,79,0.017560,0.416321,28,0.014869
4,Complete,FLOD & Bus. Controls,N/A,005056A008D11EDAA3DA9DCAF13A707E,2020-11-30,Complete,Low,Remediate,2020-11-30,3.0,...,0.00802,0.02185,Complete_Low,0.004149,0.082988,3,0.027663,0.082988,3,0.027663


### Export the data for IM:

In [ ]:
df_IM_wt_AHP.to_excel('IM_AHP_Rank_Avg.xlsx', index_label="index")